# 30分足データでモンキー戦略による損益評価

## モンキー戦略
- ランダムな買い仕掛け（乱数は設定可）
- 目標価格で手仕舞う
- 最大保有期間以内に手仕舞う

In [52]:
%matplotlib inline

import numpy as np
import numba
from numba import jit
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scipy
import scipy.stats as st
import statsmodels.stats.anova as anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import time
import datetime as dt
from dateutil.relativedelta import relativedelta
import locale
from joblib import Parallel, delayed
from functools import reduce

import finalib as fl
import finalib.mine as mi
import ta

# 月や曜日を英語で取得するためこの設定をしておく
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [53]:
# 生データ
sp_dir = 'data/e-mini-sp500-200530'
df_sp_raw = pd.read_csv(f'{sp_dir}/e-mini-sp500-30min.csv')

# 分析用データ作成

In [54]:
df = df_sp_raw.copy()

# DateTime列を追加
df['DT'] = (df['Date'] + '-' + df['Time']).map(lambda s: dt.datetime.strptime(s, '%m/%d/%Y-%H:%M'))

# 1997と2020はデータが不十分のため除く
df = df[(df['DT']>dt.datetime(1998,1,1)) & (df['DT']<dt.datetime(2020,1,1))]
df = df.reset_index(drop=True)

# 年の列を追加
df['year'] = df['DT'].map(lambda d: d.year)

# 年の順序付け
year_type = CategoricalDtype(categories=range(df['year'].min(), df['year'].max()+1), ordered=True)
df['year'] = df['year'].astype(year_type)

In [55]:
# 試験的に一部のデータだけ取り出す
df2019 = df[df['year']>=2019]
df2019 = df2019.reset_index(drop=True)
df2019 = df2019[['Open','High','Low','Close','year']]

# taのためにVolume列を追加
df2019['Vol'] = 0.0

# ta
#df2019 = ta.add_all_ta_features(df2019, open="Open", high="High", low="Low", close="Close", volume="Vol")

In [56]:
ar2019 = df2019.to_numpy().astype('float')

In [28]:
%%timeit
start = time.time()
for j, row in df2019.iterrows():
    row[3]+1
elapsed = time.time() - start
print(f'elapsed time: {elapsed:.4f} sec')
#print(f'average time: {np.average(ts):.4f} sec')

elapsed time: 3.1905 sec
elapsed time: 3.1703 sec
elapsed time: 3.1671 sec
elapsed time: 3.1519 sec
elapsed time: 3.1233 sec
elapsed time: 3.1603 sec
elapsed time: 3.1473 sec
elapsed time: 3.1278 sec
3.15 s ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
def waste_time_np(ar):
    acc = 0.0
    for j in range(ar.shape[0]):
        acc += ar[j,3]
    return acc

In [55]:
%timeit waste_time_np(ar2019)

13.5 ms ± 88.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
@jit(nopython=True)
def waste_time_numba(ar):
    acc = 0.0
    for j in range(ar.shape[0]):
        acc += ar[j,3]
    return acc

In [57]:
%timeit waste_time_numba(ar2019)

44.7 µs ± 215 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [63]:
from abc import ABC, abstractmethod, ABCMeta
from itertools import chain
from typing import Union

In [58]:
class Entry:
    def __init__(self, time, bar_idx, price):
        self.time = time
        self.bar_idx = bar_idx
        self.price = price

class Exit:
    def __init__(self, time, bar_idx, price):
        self.time = time
        self.bar_idx = bar_idx
        self.price = price

class Trade:
    def __init__(self, entry, exit, index):
        self.entry = entry
        self.exit = exit
        self.index = index

class TradeFactory:
    def __init__(self, initial=0):
        self.index = initial

    def create(self, entry, exit):
        return Trade(entry, exit, index)

class Position(ABC):
    pass

class LongPosition(Position):
    def __init__(self, price):
        self.price = price

class ShortPosition(Position):
    pass

class Trader:
    def __init__(self):
        self.trades = pd.Series()
        self.filled_entry = None
        self.current_position = None

    def add_trade(self, trade):
        self.trades[trade.index] = trade

    def order(self,)

    def entry(self, time, bar_idx, price):
        self.filled_entry = Entry(time, bar_idx, price)
        self.current_position = 

In [ ]:
sample = ar2019[:20]

class Entry:
    def __init__(self, idx, time, price):
        self.idx = idx
        self.time = time
        self.price = price

class Trader:
    def __init__(self):
        self.filled_entry = 0

    def check_entry_sign(self):
        return np.random.rand() > 0.4

    def fill_entry(self):
        self.filled_entry = 1

    def check_exit_sign(self):
        if self.filled_entry == 1:
            return np.random.rand() > 0.4

    def fill_exit(self):
        self.filled_entry = 0

history = np.zeros(sample.shape[0])
for i in range(sample.shape[0]):
    

In [12]:
class Entry:
    pass

class Exit:
    pass

class Trade:
    pass

class Trades:
    pass

class Position:
    pass

def createPositions(entry: Entry) -> [Position]:
    return []

def buy(timestamp: int, price: float, n_shares: int, positions: [Position]) -> [Position]:
    return positions

def sell(timestamp: int, price: float, n_shares: int, positions: [Position]) -> [Position]:
    return positions


bar_sequence = np.array([])
positions = []
for i in range(bar_sequence.shape[0]):
    if np.random.rand() >= 0.5:
        positions = buy(0, 100, 2, positions)

    if len(positions) > 0:
        pass
        

In [65]:
class Position:
    pass

class FilledOrder(ABC):
    @abstractmethod
    def hasPositions() -> bool:
        pass

    @abstractmethod
    def toPositions() -> [Position]:
        pass

class FilledLongEntry(FilledOrder):
    pass

class FilledBuyToCover(FilledOrder):
    pass

class Order(ABC):
    @abstractmethod
    def can_fill(self, timestamp: int, price: float) -> bool:
        # バックテストのため時間と価格の条件がそろえば任意の枚数執行できることとする
        pass

    @abstractmethod
    def fill(self, timestamp: int, price: float, n_shares: int) -> FilledOrder:
        pass

class LongEntryOrder(Order):
    pass

class BuyToCoverOrder(Order):
    pass

class Strategy(ABC):
    
    @abstractmethod
    def execute() -> Union[Order, None]:
        # 戦略の実行
        # 必要なデータを受け取って仕掛け条件が満たされているか検証する
        # 満たされていれば注文の作成と登録をする
        pass

def execute_strategies(strategies: [Strategy]) -> Union[Order, None]:
    for s in strategies:
        mayOrder = s.execute()
        if mayOrder:
            return mayOrder

    return None

def try_order(orders: [Order]) -> ([Order], [FilledOrder]):
    timestamp, price = 0, 100
    fillable_orders = [o for o in orders if o.can_fill(timestamp, price)]
    remaining_orders = [o for o in orders if not o.can_fill(timestamp, price)]
    # 本当は板情報によって執行できる枚数が決まるがここでは注文時の枚数がすべて執行されることとしている
    filled_orders = [o.fill(timestamp, price, o.n_shares)]
    return remaining_orders, filled_orders

def create_order():
    # これはTraderあるいはStrategyがもつメソッドにしておくのが良いかも
    # なぜなら売買ルールとセットだし、適切な注文（ロングorショート、売りor買い）を作る必要があるから単に関数にすると引数が多くなってしまう
    pass

bar_sequence = np.array([])
strategies = []
positions = []
orders = []
filled_orders = []
for i in range(bar_sequence.shape[0]):
    ########## 戦略の実行 ##########
    # 売買ルールの条件が満たされたら、注文を作成して注文リストに入れる
    # この条件判定はTraderあるいはStragtegyがもつメソッドで行われるはず
    mayOrder = execute_strategies(strategies)
    if mayOrder:
        orders.append(mayOrder)

    ########## 注文の執行 ##########
    # 条件がそろっていれば注文執行
    orders, current_filled_orders = try_order(orders)
    filled_orders.extend(current_filled_orders)    
    # 執行された注文の中にポジションの建つものがあればポジション保有
    positions.extend(chain.from_iterable([o.toPositions() for o in filled_orders if o.hasPositions()]))